# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


In [1]:
import pymysql

connection = pymysql.connect(host="192.168.0.45",
                            port=3306,
                            user="3team",
                            password="1111",
                            db="gas_station",
                            charset="utf8",
                           )

# try:
#     #테이블 생성
#     with connection.cursor() as cursor:
#         create_table_sql ="""
#         create table if not exists sido_accident_stats(
#             id INT AUTO_INCREMENT PRIMARY KEY,
#             sido VARCHAR(50),
#             accident_count INT,
#             death_count INT,
#             injury_count INT
#         );
#         """
#         cursor.execute(create_table_sql)
#     connection.commit()

# finally:
#     connection.close()

In [5]:
import pandas as pd

#sql 쿼리 실행하여 데이터 가져오기
query ="SELECT * FROM gas_station"
df = pd.read_sql(query, connection)

#연결종료
connection.close()

#데이터 확인
print(df.head())#처음 5개행 출력
print(df.info()) #데이터 프레임 정보출력

   station_id         station_name                  address region brand_name  \
0         422          (주)보성 세곡주유소     서울 강남구 헌릉로 731 (세곡동)    강남구      SK에너지   
1         423             오일프러스 셀프  서울 강남구 남부순환로 2651 (도곡동)    강남구      SK에너지   
2         424        지에스칼텍스㈜ 은마주유소          서울 강남구 영동대로 235    강남구      GS칼텍스   
3         425  HD현대오일뱅크㈜직영 강남셀프주유소           서울 강남구 도곡로 208    강남구   HD현대오일뱅크   
4         426  HD현대오일뱅크㈜직영 도곡셀프주유소        서울 강남구 남부순환로 2718    강남구   HD현대오일뱅크   

   gasoline_price  diesel_price self_service car_wash convenience_store  \
0            1638          1528            Y        Y                 N   
1            1679          1579            Y        Y                 N   
2            1679          1659            N        Y                 N   
3            1684          1604            Y        Y                 N   
4            1684          1589            Y        Y                 N   

  hours_24  
0        N  
1        Y  
2        N  
3        N

C:\Users\Playdata\AppData\Local\Temp\ipykernel_7812\2642228225.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


# 예제

## 테이블 생성


In [4]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""
# sql 문 마지막에 `;` 은 붙이지 않는다. --> 한번의 하나의 명령만 수행할 수 있으므로 ;가 필요없음.

In [5]:
import pymysql

try:
    conn = None # connection을 저장할 변수
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user='playdata',        # username: str
        password="1111",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송 (io작업.)
    cursor.execute("drop table if exists customer") # 1. drop
    cursor.execute(create_sql) # 2. create
    
finally:
    # 4. 연결닫기(끊기)
    if conn: # conn != None
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()   # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [9]:
from datetime import date, time, datetime
#실행시점
a = datetime.now()
print(a)
print(a.year, a.month, a.day, a.hour, a.minute, a.second) #속성 조회
b = date.today()
print(b)

# 특정 시점의 일시
c = date(2000, 10, 2)
print(c)
d = datetime(1990, 2, 7, 10, 22, 53)
print(d)
e = time(17, 22, 33)
print(e)

2025-04-08 10:12:09.843165
2025 4 8 10 12 9
2025-04-08
2000-10-02
1990-02-07 10:22:53
17:22:33


## DML

### insert


In [ ]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [ ]:
# with 문을 이용해 connection, cursor 생성: with block을 빠져 나올 때 자동으로 close() 처리한다.
#  DML(insert/update/delete) 처리 후 commit을 실행해야 영구적으로 적용된다.
with pymysql.connect(host="127.0.0.1", port=3306, user="playdata", password="1111", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql) # 반환값: 처리행수(insert/delete/update된행수, select 조회행수)
        print("처리 행수:", result)
        conn.commit() #insert/delete/update를 하고나면 commit을 해주어야 함.
   

### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [11]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")
tall = float(input("키:"))
insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, tall, "2010-01-01"])
        conn.commit()
        print("처리 행수:", result)

이름: 유관순 
이메일주소: r@a.com
키: 175.22


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [12]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [ ]:
#for data in datas;
#    cursor.execute(insert_sql, data)

In [13]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [14]:
update_sql = "update customer set tall=%s where id=%s" # 키를 변경하는 함수.
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id]) # (, placeholder들어갈 값.)
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  190.4
변경할 고객 ID:  6


처리 행수:  1


In [15]:
update_sql = "update customer set email=%s, tall=%s where id=%s" 
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 Email주소:  test@test.com
변경할 키:  174.23
변경할 고객 ID:  6


처리 행수:  1


In [16]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn: # 연결
    with conn.cursor() as cursor:                # 커서생성
        result = cursor.execute(delete_sql, [tall])
        print("처리 행수: ", result)             # 실행
        conn.commit()                           # commit

삭제기준 키: 180


처리 행수:  3


## select (DQL - Data Query Language)

-   조회결과 조회
    -   `cursor.execute("select문")` 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [17]:
sql = "select id, name, tall from customer" # db 컬럼 순이 아니라 지정한 순으로 조회.
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn: # connection 맺고
    with conn.cursor() as cursor:     # cursor생성하고
        result = cursor.execute(sql)     # connection 맺고
        print("조회행수:", result)       # 쿼리실행
        resultset = cursor.fetchall()   # select한 결과물(저장된 것)을 가져옴.

조회행수: 5


In [19]:
resultset[0][1] # 몇번쨰 데이터에 몇번째 컬럼값
# tuple(개별행 - tuple(컬럼값들 ))

'유관순 '

In [ ]:
resultset[0][1]

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [20]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 5


In [21]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}] #튜플이 아니라 list로 보여줌.

[{'id': 2, 'name': '유관순 ', 'tall': 175.22},
 {'id': 3, 'name': '김인영', 'tall': 165.0},
 {'id': 4, 'name': '오수철', 'tall': 175.0},
 {'id': 6, 'name': '김명수', 'tall': 174.23},
 {'id': 7, 'name': '이지영', 'tall': 163.0}]

In [22]:
resultset[1]['name']

'김인영'

### fetchone()


In [31]:
sql = "select * from customer where id = %s" # 특정 id 사람 조회
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [2])  #(2, )
        print("조회행수:", result)
        resultset = cursor.fetchone() #첫번째 행만
        #resultset = cursor.fetchall() #튜플안에 데이터를 줌.

조회행수: 1


In [32]:
resultset

(2,
 '유관순 ',
 'r@a.com',
 175.22,
 datetime.date(2010, 1, 1),
 datetime.datetime(2025, 4, 8, 10, 33, 2))

In [ ]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
else:
    print("조회결과가 없음.")

### fetchmany()


In [33]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개 --> 나눠서 조회하고 싶을때 사용.
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)

조회행수: 5


In [34]:
print(resultset1) 
print(resultset2)
print(resultset3)
print(resultset4)

((2, '유관순 ', datetime.date(2010, 1, 1)), (3, '김인영', datetime.date(2005, 1, 12)))
((4, '오수철', datetime.date(1995, 12, 20)), (6, '김명수', datetime.date(2000, 2, 12)))
((7, '이지영', datetime.date(1995, 4, 21)),)
()


### select문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [35]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn: # connect
    with conn.cursor() as cursor:  # cursor
        result = cursor.execute(sql) # 실행
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor: # 조회한 결과의 첫번째 결과를 tuple로 두번째를 tuple로...
            print(id, name, email, tall, birthday, created_at, sep=" , ") # 실행코드에 작성하지 않고 함수화 한다.

조회행수: 5
2 , 유관순  , r@a.com , 175.22 , 2010-01-01 , 2025-04-08 10:33:02
3 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2025-04-08 10:36:43
4 , 오수철 , def2@a.com , 175.0 , 1995-12-20 , 2025-04-08 10:36:43
6 , 김명수 , test@test.com , 174.23 , 2000-02-12 , 2025-04-08 10:36:43
7 , 이지영 , mno@abc.com , 163.0 , 1995-04-21 , 2025-04-08 10:36:43


In [5]:
import customer_db as cdb

result = cdb.select_customer_by_id(5)
print(result)

None


In [4]:
%%writefile customer_db.py

############################################33
# SQL 실행 코드를 함수화
############################################33
from datetime import date
import pymysql
def select_customer_by_id(cust_id: int) -> tuple|None: #힌트
    """
    고객 id로 고객정보를 DB에서 조회해서 반환하는 함수.
    Args:
    Returns:
        tuple : 조회결과
        None : 조회결과가 없을 경우
    Raises:
    """
    sql = "select * from customer where id = %s" 
    with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cursor.execute(sql, [cust_id])  
            return  cursor.fetchone()


def select_all_customer():
    """
    전체 고객정보를 조회하는 함수
    select * from customer;
    """
    pass


def update_customer(cust_id:int, name:str, email:str, tall:float, birthday:date|str) -> int:

    sql = ("update customerset "
           "set name=%s, email=%s, tall=%s, birthday=%s"
           "where id=%s")
    with pymysql.connect(host="127.0.0.1", port=3306, user='playdata', password='1111', db='mydb') as conn:
        with conn.cursor() as cursor:
            result = cusor.execute(sql, (name, email, tall, birthday, cust_id))
            conn.commit()
            return result

Overwriting customer_db.py


In [ ]:
update_customer(5,"rlaalsdud")

In [41]:
result = select_customer_by_id(5)
print(result)

None


In [42]:
"aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa\
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" # 두줄이지만 한줄인 것 처럼 '\'는 연결되어있음을 표시. 공백을 넣으면 공백이 그대로 표시됨.

# ("aaaaaaaaaaaaaaaaaaaaaaaaaaa"
#"bbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"
#"ccccccccccccccccccccccccccccccc") # 한줄이지만 길어서 나눈거니깐 보여줄땐 한줄로 보여줘.

'aaaaaaaaaaaaaaaaaaaaaaaaaaabbbbbbbbbbbbbbbbbbbbbbbbbbbbbbccccccccccccccccccccccccccccccc'